## Reading Plate Number using OCR

In [2]:
import cv2
import easyocr
import re
import os
from ultralytics import YOLO

In [3]:
# Load model YOLO hasil training
model_path = r"training_results\weights\best.pt"
model = YOLO(model_path)

In [5]:
# Inisialisasi OCR
reader = easyocr.Reader(['en', 'id'])  # Bisa tambahkan 'id' jika ada model bahasa Indonesia

Progress: |██████████████████████████████████████████████████| 100.0% Complete

In [12]:
# Path gambar yang ingin diproses
image_path = r"test_input\test.jpg"
output_folder = r"test_reading"

In [13]:
# Pastikan folder output ada
os.makedirs(output_folder, exist_ok=True)

In [14]:
# Load gambar
image = cv2.imread(image_path)

In [15]:

# Jalankan YOLO untuk deteksi bounding box plat nomor
results = model(image)

# Loop melalui hasil deteksi
for i, result in enumerate(results):
    for j, box in enumerate(result.boxes):
        # Ambil koordinat bounding box
        x1, y1, x2, y2 = map(int, box.xyxy[0])  

        # Crop plat nomor dari gambar asli
        plate_crop = image[y1:y2, x1:x2]

        # Convert ke grayscale untuk OCR
        plate_gray = cv2.cvtColor(plate_crop, cv2.COLOR_BGR2GRAY)

        # Simpan hasil crop
        cropped_plate_path = os.path.join(output_folder, f"cropped_plate_{i}_{j}.jpg")
        cv2.imwrite(cropped_plate_path, plate_crop)

        print(f"Hasil crop plat nomor disimpan di: {cropped_plate_path}")

        # Jalankan OCR
        ocr_results = reader.readtext(plate_gray)

        # Gabungkan hasil OCR menjadi teks
        detected_text = " ".join([text[1] for text in ocr_results])

        # Gunakan regex untuk mencari nomor plat dan tanggal pajak
        plat_nomor_pattern = r"[A-Z]{1,2} \d{1,4} [A-Z]{1,3}"  # Format plat nomor
        pajak_pattern = r"\d{2}\.\d{2}"  # Format tanggal pajak MM.YY

        plat_nomor = re.search(plat_nomor_pattern, detected_text)
        pajak = re.search(pajak_pattern, detected_text)

        # Cetak hasil
        print(f"Nomor Plat: {plat_nomor.group() if plat_nomor else 'Tidak ditemukan'}")
        print(f"Tanggal Pajak: {pajak.group() if pajak else 'Tidak ditemukan'}")




0: 480x640 1 plat_nomor, 272.0ms
Speed: 6.0ms preprocess, 272.0ms inference, 145.1ms postprocess per image at shape (1, 3, 480, 640)
Hasil crop plat nomor disimpan di: test_reading\cropped_plate_0_0.jpg
Nomor Plat: B 1627 PAJ
Tanggal Pajak: Tidak ditemukan


In [16]:
print("Hasil OCR Mentah:", ocr_results)


Hasil OCR Mentah: [([[10, 0], [339, 0], [339, 93], [10, 93]], 'B 1627 PAJ]', 0.4430586876887411), ([[128, 82], [170, 82], [170, 112], [128, 112]], '09', 0.9992297268215895), ([[134, 108], [220, 108], [220, 133], [134, 133]], 'OMw Wuna', 0.12073781686210408), ([[163.65835921350012, 80.31671842700025], [217.504318037472, 76.34821576251848], [218.34164078649988, 105.68328157299975], [165.495681962528, 110.65178423748152]], "'25", 0.7175507815418156)]


In [17]:
# Gabungkan hasil OCR jadi satu teks
detected_text = " ".join([text[1] for text in ocr_results])

# Bersihkan teks dari karakter yang mengganggu (' atau lainnya)
detected_text = re.sub(r"[^\w\s\.\-]", "", detected_text)

# Gunakan regex yang lebih fleksibel
pajak_pattern = r"\d{2}[\.\,\-\s]?\d{2}"


In [18]:
print(f"Teks OCR setelah pembersihan: {detected_text}")


Teks OCR setelah pembersihan: B 1627 PAJ 09 OMw Wuna 25


In [19]:
pajak = re.search(pajak_pattern, detected_text)
print(f"Tanggal Pajak: {pajak.group() if pajak else 'Tidak ditemukan'}")


Tanggal Pajak: 1627


In [20]:
import re

# OCR output setelah pembersihan
detected_text = "B 1627 PAJ 09 OMw Wuna 25"

# Cari semua angka dengan 2 digit
matches = re.findall(r"\b\d{2}\b", detected_text)

# Jika ditemukan minimal 2 angka, asumsikan MM.YY dari OCR
if len(matches) >= 2:
    pajak = f"{matches[0]}.{matches[1]}"
else:
    pajak = "Tidak ditemukan"

print(f"Tanggal Pajak: {pajak}")


Tanggal Pajak: 09.25


In [21]:
image_path_2 = r"test_input\test2.jpeg"

In [22]:
# Load gambar
image = cv2.imread(image_path_2)

In [23]:

# Jalankan YOLO untuk deteksi bounding box plat nomor
results = model(image)

# Loop melalui hasil deteksi
for i, result in enumerate(results):
    for j, box in enumerate(result.boxes):
        # Ambil koordinat bounding box
        x1, y1, x2, y2 = map(int, box.xyxy[0])  

        # Crop plat nomor dari gambar asli
        plate_crop = image[y1:y2, x1:x2]

        # Convert ke grayscale untuk OCR
        plate_gray = cv2.cvtColor(plate_crop, cv2.COLOR_BGR2GRAY)

        # Simpan hasil crop
        cropped_plate_path = os.path.join(output_folder, f"cropped_plate_{i}_{j}.jpg")
        cv2.imwrite(cropped_plate_path, plate_crop)

        print(f"Hasil crop plat nomor disimpan di: {cropped_plate_path}")

        # Jalankan OCR
        ocr_results = reader.readtext(plate_gray)

        # Gabungkan hasil OCR menjadi teks
        detected_text = " ".join([text[1] for text in ocr_results])

        # Gunakan regex untuk mencari nomor plat dan tanggal pajak
        plat_nomor_pattern = r"[A-Z]{1,2} \d{1,4} [A-Z]{1,3}"  # Format plat nomor
        pajak_pattern = r"\d{2}\.\d{2}"  # Format tanggal pajak MM.YY

        plat_nomor = re.search(plat_nomor_pattern, detected_text)
        pajak = re.search(pajak_pattern, detected_text)

        # Cetak hasil
        print(f"Nomor Plat: {plat_nomor.group() if plat_nomor else 'Tidak ditemukan'}")
        print(f"Tanggal Pajak: {pajak.group() if pajak else 'Tidak ditemukan'}")




0: 480x640 1 plat_nomor, 258.6ms
Speed: 7.0ms preprocess, 258.6ms inference, 5.0ms postprocess per image at shape (1, 3, 480, 640)
Hasil crop plat nomor disimpan di: test_reading\cropped_plate_0_0.jpg
Nomor Plat: B 2156 TOR
Tanggal Pajak: 09.27


In [26]:
image_path_3 = r"test_input\test4.jpeg"

In [27]:
# Load gambar
image = cv2.imread(image_path_3)

In [28]:

# Jalankan YOLO untuk deteksi bounding box plat nomor
results = model(image)

# Loop melalui hasil deteksi
for i, result in enumerate(results):
    for j, box in enumerate(result.boxes):
        # Ambil koordinat bounding box
        x1, y1, x2, y2 = map(int, box.xyxy[0])  

        # Crop plat nomor dari gambar asli
        plate_crop = image[y1:y2, x1:x2]

        # Convert ke grayscale untuk OCR
        plate_gray = cv2.cvtColor(plate_crop, cv2.COLOR_BGR2GRAY)

        # Simpan hasil crop
        cropped_plate_path = os.path.join(output_folder, f"cropped_plate_{i}_{j}.jpg")
        cv2.imwrite(cropped_plate_path, plate_crop)

        print(f"Hasil crop plat nomor disimpan di: {cropped_plate_path}")

        # Jalankan OCR
        ocr_results = reader.readtext(plate_gray)

        # Gabungkan hasil OCR menjadi teks
        detected_text = " ".join([text[1] for text in ocr_results])

        # Gunakan regex untuk mencari nomor plat dan tanggal pajak
        plat_nomor_pattern = r"[A-Z]{1,2} \d{1,4} [A-Z]{1,3}"  # Format plat nomor
        pajak_pattern = r"\d{2}\.\d{2}"  # Format tanggal pajak MM.YY

        plat_nomor = re.search(plat_nomor_pattern, detected_text)
        pajak = re.search(pajak_pattern, detected_text)

        # Cetak hasil
        print(f"Nomor Plat: {plat_nomor.group() if plat_nomor else 'Tidak ditemukan'}")
        print(f"Tanggal Pajak: {pajak.group() if pajak else 'Tidak ditemukan'}")




0: 480x640 1 plat_nomor, 300.3ms
Speed: 1866.8ms preprocess, 300.3ms inference, 57.3ms postprocess per image at shape (1, 3, 480, 640)
Hasil crop plat nomor disimpan di: test_reading\cropped_plate_0_0.jpg
Nomor Plat: Tidak ditemukan
Tanggal Pajak: Tidak ditemukan
